In [1]:
from app import create_app
from app.models import *
from app.utils import *
from datetime import datetime as dt
from dotenv import  load_dotenv
import os
from random import randint
import dotenv
dotenv.load_dotenv()

app = create_app()

## User

### Create admin, jane_doe and 9 test users

In [ ]:
with app.app_context():
    objs_to_add = []
    
    # Code Block
    #################################################
    
    if not User.query.filter_by(username='admin').first():
        # add admin
        admin = User(username = 'admin', email = 'admin@example.com', password = "a234567!")
        objs_to_add.append(admin)
    else: pass
    
    if not User.query.filter_by(username='seb').first():
        jane_doe = User(username = 'jane_doe', email = 'jane_doe@example.com', password = "a234567!")
        objs_to_add.append(jane_doe)
    else: pass
    
    for idx, user in enumerate(range(9)):
        random_num = randint(1000, 10000000)
        username = f'test_{random_num}'
        u = User(username = username, email = f'{username}@example.com', password = "a234567!")
        objs_to_add.append(u)
    
    ##################################################
    db.session.add_all(objs_to_add)
    add_to_db, msg = commit_to_db(db)
    print(msg)
    db.session.close()




### Create admin and user roles

In [ ]:
with app.app_context():
    objs_to_add = []
    
    # Code Block
    #################################################
    
    admin = Role(name = 'admin', description = 'Administrator')
    user = Role(name = 'user', description = 'User')
    objs_to_add.extend([admin, user])
    
    
    ##################################################
    db.session.add_all(objs_to_add)
    _, msg = commit_to_db(db)
    print(msg)
    db.session.close()

### Assign user roles

In [ ]:
with app.app_context():
    objs_to_add = []
    
    # Code Block
    #################################################
    
    user_admin = User.query.filter_by(username="admin").first()
    all_users = User.query.filter(User.username!= "admin").all()
    role_admin = Role.query.filter_by(name="admin").first()
    role_user = Role.query.filter_by(name="user").first()
    objs_to_add.append(UserRole(user=user_admin, role=role_admin))
    
    for u in all_users:
        objs_to_add.append(UserRole(user=u, role=role_user))
    
    ##################################################  
    db.session.add_all(objs_to_add)
    _, msg = commit_to_db(db)
    print(msg)
    db.session.close()

## Persona

### Create 2 Personas

In [ ]:
with app.app_context():
    objs_to_add = []
    
    # Code Block
    #################################################
    
    unknown, bro =  Persona(name="Unknown", persona_id=1), Persona(name="Bro") 
    objs_to_add = [unknown, bro]
    
    ##################################################
    db.session.add_all(objs_to_add)
    _, msg = commit_to_db(db)
    print(msg)
    db.session.close()

## Advice

### Add two advice slips from "Unknown" persona

In [2]:
with app.app_context():
    objs_to_add = []
    
    # Code Block
    #################################################
    
    # Advice is a child of Entity, so first create an Entity.
    advice_entity_1 = Entity(type = "advice")
    advice_entity_2 = Entity(type = "advice")
    objs_to_add.extend([advice_entity_1, advice_entity_2])
    
    # Proceed only if the entity is saved successfully.
    persona = Persona.query.filter_by(name = "Unknown").first()
    advice1 = Advice(
        entity = advice_entity_1, 
        persona = persona,
        content = "Remember that spiders are more afraid of you, than you are of them.",
        adviceslip_id = 1)
    advice2 = Advice(
        entity = advice_entity_2, 
        persona = persona,
        content = "The more ideas that you give away, the more ideas that will come to you.",
        adviceslip_id = 54)
    objs_to_add.extend([advice1, advice2])            
    
    ##################################################
    db.session.add_all(objs_to_add)
    saved_to_db, msg = commit_to_db(db)
    if saved_to_db: print(msg)
    else: raise msg
    db.session.close()

success


### User 'jane_doe' likes the advice about spiders.

In [ ]:
with app.app_context():
    objs_to_add = []
    
    # Code Block
    #################################################
    
    advice_spider = Advice.query.filter(Advice.content.op('~*')('[Sp]pider')).first()
    jane_doe = User.query.filter_by(username = 'jane_doe').first()
    view_entity = EntityView(entity =advice_spider.entity, user = jane_doe)
    like_entity = EntityLike(entity =advice_spider.entity, user = jane_doe)
    objs_to_add.extend([like_entity, view_entity])

    ##################################################
    db.session.add_all(objs_to_add)
    saved_to_db, msg = commit_to_db(db)
    if saved_to_db: print(msg)
    else: raise msg
    db.session.close()

## Tags

### Add a 50 categories (with their descriptions) to Tag table

In [ ]:
categories = ["Relationships","Career","Finance","Development","Parenting","Health","Travel","Education","Home","Law","Communication","Fashion","Technology","Food","Environment","Spirituality","Safety","Pets","Photography","Hobbies","Marketing","Volunteering","Public speaking","Stress","Writing","Organization","Language","Dating","Branding","Transportation", "Gardening","DIY","Shopping","Insurance","Gambling","Workplace","Networking","Investing","Retirement","Family","Etiquette","Wedding","Funerals","Emotions","Disability","Drugs","Grief","Mental Health","Elderly", "Cybersecurity"]
descriptions = ["Advice on maintaining and improving romantic relationships", "Advice on career development, job search, and professional growth", "Advice on managing personal finances, budgeting, and investing","Advice on personal growth and self-improvement", "Advice on raising children and parenting strategies", "Advice on maintaining physical and mental well-being","Advice on planning and enjoying travel experiences","Advice on pursuing education and achieving academic success","Advice on home improvement, decorating, and maintenance","Advice on legal issues and rights","Advice on building and maintaining relationships and social skills","Advice on personal style and grooming","Advice on using and understanding technology","Advice on cooking, nutrition, and meal planning","Advice on protecting the environment and living sustainably","Advice on spirituality, religion, and personal beliefs","Advice on personal safety, security and emergency preparedness","Advice on caring for and training domestic animals","Advice on capturing and creating photographs and videos","Advice on pursuing and enjoying hobbies and interests","Advice on building and promoting a personal or business brand","Advice on giving back to the community and volunteering","Advice on public speaking, presentations and influencing others","Advice on dealing with stress and relaxation techniques","Advice on writing, publishing and storytelling","Advice on personal organization, productivity and time management","Advice on learning new languages","Advice on online dating and relationships","Advice on personal branding and online reputation management","Advice on choosing and maintaining vehicles, and alternative transportation options", "Advice on cultivating plants, lawns, and gardens", "Advice on doing home repairs and DIY projects","Advice on smart shopping and consumer rights","Advice on understanding and purchasing insurance policies","Advice on gambling, games of chance and online betting","Advice on navigating office politics and building professional relationships in the workplace","Advice on building a professional network and making connections","Advice on investing in stocks, real estate, and other assets","Advice on preparing for retirement and managing retirement funds","Advice on maintaining family relationships and dynamics","Advice on manners and etiquette for different social situations","Advice on planning and organizing weddings","Advice on navigating and coping with funerals","Advice on understanding and managing emotions","Advice on navigating and coping with disability","Advice on understanding and managing drug addiction","Advice on coping with grief and loss","Advice on understanding and managing mental health issues","Advice on understanding and managing aging and elderly care", "Advice on protecting personal and business information online"]

with app.app_context():
    objs_to_add = []
    
    # Code Block
    #################################################
    
    for category, description in zip(categories, descriptions):
        tag = Tag(name=category, description=description)
        objs_to_add.append(tag)
    
    ##################################################
    db.session.add_all(objs_to_add)
    saved_to_db, msg = commit_to_db(db)
    if saved_to_db: print(msg)
    else: raise msg
    db.session.close()

### Tag the advice about spiders with the "Home" and "Safety" tags.

In [ ]:
with app.app_context():
    objs_to_add = []
    
    # Code Block
    #################################################
    
    tag_home = Tag.query.filter_by(name = 'Home').first()
    tag_safety = Tag.query.filter_by(name = 'Safety').first()
    advice_spider = Advice.query.filter(Advice.content.op('~*')('[Sp]pider')).first()
    entity_tag_home = EntityTag(entity = advice_spider.entity, tag = tag_home)
    entity_tag_safety = EntityTag(entity = advice_spider.entity, tag = tag_safety)
    objs_to_add.extend([entity_tag_home, entity_tag_safety])
    
    ##################################################
    db.session.add_all(objs_to_add)
    saved_to_db, msg = commit_to_db(db)
    if saved_to_db: print(msg)
    else: raise msg
    db.session.close()

## Comments

### jane_doe comments on advice about spiders

In [ ]:
with app.app_context():
    objs_to_add = []
    
    # Code Block
    #################################################
    
    advice_spider = Advice.query.filter(Advice.content.op('~*')('[Sp]pider')).first()
    jane_doe = User.query.filter_by(username = 'jane_doe').first()
    comment = EntityComment(entity = advice_spider.entity, user = jane_doe, content="This is helpful advice.")
    objs_to_add.append(comment)

    ##################################################
    db.session.add_all(objs_to_add)
    saved_to_db, msg = commit_to_db(db)
    if saved_to_db: print(msg)
    else: raise msg
    db.session.close()

### Admin likes jane_doe's comment

In [ ]:
with app.app_context():
    objs_to_add = []
    
    # Code Block
    #################################################
    
    advice_spider = Advice.query.filter(Advice.content.op('~*')('[Sp]pider')).first()
    jane_doe = User.query.filter_by(username = 'jane_doe').first()
    comment = EntityComment.query.filter_by(user_id = jane_doe.user_id, entity_id = advice_spider.entity_id, content = "This is helpful advice.").first()
    like_comment = EntityCommentLike(comment= comment, user = jane_doe)
    objs_to_add.append(like_comment)

    ##################################################
    db.session.add_all(objs_to_add)
    saved_to_db, msg = commit_to_db(db)
    if saved_to_db: print(msg)
    else: raise msg
    db.session.close()

### Get count of comment's likes

In [2]:
with app.app_context():

    
    # Code Block
    #################################################
    
    advice_spider = Advice.query.filter(Advice.content.op('~*')('[Sp]pider')).first()
    jane_doe = User.query.filter_by(username = 'jane_doe').first()
    comment = EntityComment.query.filter_by(user_id = jane_doe.user_id, entity_id = advice_spider.entity_id, content = "This is helpful advice.").first()
    print(comment.likes())

    ##################################################


1


In [5]:
import openai

## OpenAI Example Usage

In [3]:
import os
import requests
import dotenv
dotenv.load_dotenv()

openai_url = os.getenv('OPENAI_BASE_API_URL')
openai_apikey = os.getenv('OPENAI_API_KEY')
model = os.getenv('OPENAI_FINETUNED_MODEL')

prompt = "Every tree must be climbed from the bottom:::"
url = openai_url + "/completions"

payload = {
    "prompt": prompt,
    "model": model,
    "max_tokens": 1024,
    "stop": ":::",  # special stop token for my finetuned model.
    "temperature": 0.2
}

headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {openai_apikey}"
}

response = requests.post(url, json=payload, headers=headers)

print(response.json())

{'id': 'cmpl-6a8BYdw28Yc6jBUH6PIlodVyZUL3B', 'object': 'text_completion', 'created': 1674069292, 'model': 'davinci:ft-personal-2023-01-18-05-54-06', 'choices': [{'text': ' Climb that tree, bros! Go big or go home. Every tree must be climbed from the bottom, like a ladder to success. Take the initiative and start at the bottom and work your way to the top.', 'index': 0, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 10, 'completion_tokens': 45, 'total_tokens': 55}}


### alternative 

In [7]:
import os
import dotenv
import openai
dotenv.load_dotenv()

openai_url = os.getenv('OPENAI_BASE_API_URL')
openai_apikey = os.getenv('OPENAI_API_KEY')
model = os.getenv('OPENAI_FINETUNED_MODEL')

openai.Completion.create(
    model=model, 
    prompt = "Some things are better left unsaid:::",
    temperature = 0.2,
    stop = [":::"],
    max_tokens= 1024,
    
)

<OpenAIObject text_completion id=cmpl-6aCmW1eyaj3341Pp2g188ejDowkG0 at 0x266d18b7310> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": " Don't be a chatterbox, bros. Some things are better left unsaid. Keep some secrets, surprise people with the unexpected, and maintain that mysterious bro-factor."
    }
  ],
  "created": 1674086960,
  "id": "cmpl-6aCmW1eyaj3341Pp2g188ejDowkG0",
  "model": "davinci:ft-personal-2023-01-18-05-54-06",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 36,
    "prompt_tokens": 9,
    "total_tokens": 45
  }
}

## Advice Slip Example Usage

In [5]:
import requests

url = 'https://api.adviceslip.com/advice/225'

response = requests.get(url)

print(response.json())
"slip" in response.json()


{'message': {'type': 'error', 'text': 'Advice slip not found.'}}


False